# Questão 1

## A nota geral do ENADE dos cursos é influenciada pelo custo aluno? Isto é, quanto mais caro o aluno, mais investimento é feito e maior é a nota?

In [1]:
import numpy as np
import pandas as pd

In [15]:
# Carregando os dados
ies_custo = pd.read_csv('ENADE2017/dados/instituicoes_federais_nordeste_custo.csv')
ies_custo

,Instituicao,Sigla,despesaLiquidada,totalAlunos,custoAluno
0,UNIVERSIDADE FEDERAL DE PERNAMBUCO (UFPE),UFPE,1557.9,38774,39754.9
1,UNIVERSIDADE FEDERAL DE ALAGOAS (UFAL),UFAL,802.8,23741,33813.6
2,UNIVERSIDADE FEDERAL DA BAHIA (UFBA),UFBA,1480.0,41445,35709.2
3,UNIVERSIDADE FEDERAL DE CAMPINA GRANDE (UFCG),UFCG,661.4,17194,38468.7
4,UNIVERSIDADE FEDERAL RURAL DE PERNAMBUCO (UFRPE),UFRPE,579.0,13989,41387.5
5,UNIVERSIDADE FEDERAL DO CEARÁ (UFC),UFC,1560.1,32269,48346.1
6,UNIVERSIDADE FEDERAL DO PIAUÍ (UFPI),UFPI,776.2,30348,25575.5
7,UNIVERSIDADE FEDERAL DA PARAÍBA (UFPB),UFPB,1551.5,34084,45518.8
8,UNIVERSIDADE FEDERAL DO MARANHÃO (UFMA),UFMA,729.5,32204,22652.6
9,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE (U...,UFRN,1694.5,32872,51548.1


In [8]:
resultados_enade_tudo = pd.read_csv('ENADE2017/dados/resultados_conceito_enade_2017.csv')
resultados_enade_tudo.info(verbose = True, null_counts = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10570 entries, 0 to 10569
Data columns (total 22 columns):
Ano                                 int64
Código da Área                      int64
Área de Avaliação                   object
Código da IES                       int64
Nome da IES                         object
Sigla da IES                        object
Organização Acadêmica               object
Categoria Administrativa            object
Código do Curso                     int64
Modalidade de Ensino                object
Código do Município                 int64
Município do Curso                  object
Sigla da UF                         object
Nº de Concluintes Inscritos         int64
Nº  de Concluintes Participantes    int64
Nota Bruta - FG                     object
Nota Padronizada - FG               object
Nota Bruta - CE                     object
Nota Padronizada - CE               object
Conceito Enade (Contínuo)           object
Conceito Enade (Faixa)              ob

In [9]:
resultados_enade_tudo

,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,Modalidade de Ensino,...,Sigla da UF,Nº de Concluintes Inscritos,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação
0,2017,21,ARQUITETURA E URBANISMO,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,44,Educação Presencial,...,MT,44,43,"59,4977","3,1401","62,9744","4,4030","4,0873",5,NaN
1,2017,21,ARQUITETURA E URBANISMO,2,UNIVERSIDADE DE BRASÍLIA,UNB,Universidade,Pública Federal,132,Educação Presencial,...,DF,141,128,"61,3602","3,4161","60,7516","4,1266","3,9490",5,NaN
2,2017,21,ARQUITETURA E URBANISMO,2,UNIVERSIDADE DE BRASÍLIA,UNB,Universidade,Pública Federal,1138293,Educação Presencial,...,DF,123,110,"58,3155","2,9649","55,6845","3,4965","3,3636",4,NaN
3,2017,21,ARQUITETURA E URBANISMO,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,Universidade,Pública Federal,99444,Educação Presencial,...,SE,42,31,"62,2290","3,5448","57,5161","3,7243","3,6794",4,NaN
4,2017,21,ARQUITETURA E URBANISMO,4,UNIVERSIDADE FEDERAL DO AMAZONAS,UFAM,Universidade,Pública Federal,122640,Educação Presencial,...,AM,64,50,"60,3600","3,2679","54,7100","3,3753","3,3485",4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10565,2017,6409,TECNOLOGIA EM GESTÃO DA TECNOLOGIA DA INFORMAÇÃO,15803,FACULDADE DE TECNOLOGIA DE TATUÍ - PROF. WILSO...,FATEC TATUÍ,Faculdade,Pública Estadual,1156898,Educação Presencial,...,SP,59,44,"47,8159","2,9689","47,4000","3,0920","3,0612",4,NaN
10566,2017,6409,TECNOLOGIA EM GESTÃO DA TECNOLOGIA DA INFORMAÇÃO,16410,Faculdade de Tecnologia de Catanduva,FATEC Catanduva,Faculdade,Pública Estadual,1284534,Educação Presencial,...,SP,31,25,"56,1200","4,3336","52,0240","3,9114","4,0170",5,NaN
10567,2017,6409,TECNOLOGIA EM GESTÃO DA TECNOLOGIA DA INFORMAÇÃO,16894,Faculdade Internacional de São Luís,ISL,Faculdade,Privada com fins lucrativos,1157132,Educação Presencial,...,MA,49,9,"54,2444","4,0254","51,4000","3,8009","3,8570",4,NaN
10568,2017,6409,TECNOLOGIA EM GESTÃO DA TECNOLOGIA DA INFORMAÇÃO,17277,Faculdade SENAC Criciúma,Senac Criciúma,Faculdade,Privada sem fins lucrativos,5000999,Educação Presencial,...,SC,1,1,"25,2000",NaN,"28,8000",NaN,NaN,SC,NaN


In [5]:
ies_brasil_emec = pd.read_csv('ENADE2017/dados/ies_brasil_emec_set_2019.csv')
ies_brasil_emec.info(verbose = True, null_counts = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 21 columns):
Código Mantenedora          int64
Razão Social                object
CNPJ                        object
Natureza Jurídica           object
Código IES                  int64
Instituição(IES)            object
Sigla                       object
Endereço                    object
Município                   object
UF                          object
Organização Acadêmica       object
Tipo de Credenciamento      object
Categoria                   object
Categoria Administrativa    object
CI                          object
Ano CI                      float64
CI-EaD                      object
Ano CI-EaD                  float64
IGC                         object
Ano IGC                     float64
Situação                    object
dtypes: float64(3), int64(2), object(16)
memory usage: 523.8+ KB


In [12]:
microdados = pd.read_csv('ENADE2017/dados/microdados_ENADE_2017.csv')
microdados

,,,,,,,NU_ANO;CO_IES;CO_CATEGAD;CO_ORGACAD;CO_GRUPO;CO_CURSO;CO_MODALIDADE;CO_MUNIC_CURSO;CO_UF_CURSO;CO_REGIAO_CURSO;NU_IDADE;TP_SEXO;ANO_FIM_EM;ANO_IN_GRAD;CO_TURNO_GRADUACAO;TP_INSCRICAO_ADM;TP_INSCRICAO;NU_ITEM_OFG;NU_ITEM_OFG_Z;NU_ITEM_OFG_X;NU_ITEM_OFG_N;NU_ITEM_OCE;NU_ITEM_OCE_Z;NU_ITEM_OCE_X;NU_ITEM_OCE_N;DS_VT_GAB_OFG_ORIG;DS_VT_GAB_OFG_FIN;DS_VT_GAB_OCE_ORIG;DS_VT_GAB_OCE_FIN;DS_VT_ESC_OFG;DS_VT_ACE_OFG;DS_VT_ESC_OCE;DS_VT_ACE_OCE;TP_PRES;TP_PR_GER;TP_PR_OB_FG;TP_PR_DI_FG;TP_PR_OB_CE;TP_PR_DI_CE;TP_SFG_D1;TP_SFG_D2;TP_SCE_D1;TP_SCE_D2;TP_SCE_D3;NT_GER;NT_FG;NT_OBJ_FG;NT_DIS_FG;NT_FG_D1;NT_FG_D1_PT;NT_FG_D1_CT;NT_FG_D2;NT_FG_D2_PT;NT_FG_D2_CT;NT_CE;NT_OBJ_CE;NT_DIS_CE;NT_CE_D1;NT_CE_D2;NT_CE_D3;CO_RS_I1;CO_RS_I2;CO_RS_I3;CO_RS_I4;CO_RS_I5;CO_RS_I6;CO_RS_I7;CO_RS_I8;CO_RS_I9;QE_I01;QE_I02;QE_I03;QE_I04;QE_I05;QE_I06;QE_I07;QE_I08;QE_I09;QE_I10;QE_I11;QE_I12;QE_I13;QE_I14;QE_I15;QE_I16;QE_I17;QE_I18;QE_I19;QE_I20;QE_I21;QE_I22;QE_I23;QE_I24;QE_I25;QE_I26;QE_I27;QE_I28;QE_I29;QE_I30;QE_I31;QE_I32;QE_I33;QE_I34;QE_I35;QE_I36;QE_I37;QE_I38;QE_I39;QE_I40;QE_I41;QE_I42;QE_I43;QE_I44;QE_I45;QE_I46;QE_I47;QE_I48;QE_I49;QE_I50;QE_I51;QE_I52;QE_I53;QE_I54;QE_I55;QE_I56;QE_I57;QE_I58;QE_I59;QE_I60;QE_I61;QE_I62;QE_I63;QE_I64;QE_I65;QE_I66;QE_I67;QE_I68;QE_I69;QE_I70;QE_I71;QE_I72;QE_I73;QE_I74;QE_I75;QE_I76;QE_I77;QE_I78;QE_I79;QE_I80;QE_I81
2017;1;1;10028;5710;3;1;5103403;51;5;26;F;2007;2012;3;0;1;8;0;0;0;27;0;8;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDDDCBCBBEADDCBEDB;EDXXEABCXBXDDCXCXBEADXCBEXB;DCBBCEAD;01111111;EEEADDBCABADCCAAAEEDDDCAACB;109900119191019090101910091;555;555;555;555;555;555;555;333;555;555;333;52,6;64,1;87,5;29,0;58;50;60;0;0;0;48,7;52,6;26,7;55;25;0;C;D;D;B;C;C;B;D;C;A;A;A;D;F;A;A;B;C;...
2017;1;1;10028;5710;3;1;5103403;51;5;23;F;2013;2013;3;0;1;8;0;0;0;27;0;8;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDDDCBCBBEADDCBEDB;EDXXEABCXBXDDCXCXBEADXCBEXB;CCBBCEBD;11111101;ECDEAADCABADDCDEAADCDCCBECB;109901019191119090001911191;555;555;555;555;555;555;555;555;555;555;555;63,1;82,1;87,5;74,0;72;60;75;76;80;75;56,7;63,2;20,0;60;0;0;D;E;A;C;D;D;A;B;D;B;D;A;E;D;C;B;B;D;E...
2017;1;1;10028;5710;3;1;5103403;51;5;23;M;2011;2013;3;0;1;8;0;0;0;27;0;8;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDDDCBCBBEADDCBEDB;EDXXEABCXBXDDCXCXBEADXCBEXB;CCDBCEAD;11011111;EADEEAAACAAECDACDEEBCCECBEA;109911009090009190100900090;555;555;555;333;555;555;333;333;555;333;333;33,7;52,5;87,5;0,0;0;0;0;0;0;0;27,4;26,3;33,3;100;0;0;D;B;A;B;B;B;D;D;A;A;D;A;E;F;B;E;F;C;...
2017;1;1;10028;5710;3;1;5103403;51;5;23;M;2011;2013;3;0;1;8;0;0;0;27;0;8;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDDDCBCBBEADDCBEDB;EDXXEABCXBXDDCXCXBEADXCBEXB;CABBCEBD;10111101;BEDDEABECBADDACCEACBDECAEAB;009911109191109190001910191;555;555;555;555;555;555;555;555;555;555;555;58,1;72,6;75,0;69,0;67;75;65;71;75;70;53,2;57,9;26,7;80;0;0;B;C;C;B;B;C;D;D;B;A;A;A;D;D;D;A;B;C;A...
2017;1;1;10028;5710;3;1;5103403;51;5;24;M;2010;2013;3;0;1;8;0;0;0;27;0;8;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDDDCBCBBEADDCBEDB;EDXXEABCXBXDDCXCXBEADXCBEXB;CCBBCDAE;11111010;EDDDDDBEAABDDDAECBBBEEEDAAB;119900109091109091000900091;555;555;555;555;555;555;555;555;555;555;555;46,3;73,2;75,0;70,5;75;75;75;66;50;70;37,3;36,8;40,0;0;100;20;C;C;C;B;B;C;D;D;B;A;A;A;D;F;B;B;F;C...
...,...,...,...,...,...,...,...
2017;19578;2;10022;6208;5001279;1;3513009;35;3;26;M;2013;2014;4;0;1;8;0;0;0;27;1;4;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDCADEDDECEBABCABZ;EDXXEABCXBDCADEXDECEBABCABZ;;;;;888;888;888;888;888;888;888;888;888;888;888;;;;;;;;;;;;;;;;;.;.;.;.;.;.;.;.;.;A;D;A;B;C;B;D;C;E;E;A;A;A;A;D;35;A;A;B;C;B;B;B;A;A;F;5;7;7;4;4;5;4;5;4;7;4;4;4;3;3;4;4;3;4;4;3;3;4;4;4;7;7;7;4;4;4;3;4;4;3;3;3;4;3;4;4;3;;;;;;;;;;;;;,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017;19578;2;10022;6208;5001279;1;3513009;35;3;41;M;2013;2014;4;0;1;8;0;0;0;27;1;4;0;CCBBCEAD;CCBBCEAD;EDCAEABCDBDCADEDDECEBABCABZ;EDXXEABCXBDCADEXDECEBABCABZ;;;;;888;888;888;888;888;888;888;888;888;888;888;;;;;;;;;;;;;;;;;.;.;.;.;.;.;.;.;.;B;A;A;B;D;C;C;B;E;A;A;A;A;A;A;35;A;A;A;G;A;B;D;A;C;F;5;5;4;3;5;4;5;4;3;5;3;3;3;3;4;4;5;5;5;2;4;4;5;2;5;8;3;4;3;2;2;4;